In [ ]:
# Install Audio Control (for volume)
!pip install comtypes pycaw

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/246.4 kB ? eta -:--:--
     ---- ---------------------------------- 30.7/246.4 kB 1.3 MB/s eta 0:00:01
     ---- ---------------------------------- 30.7/246.4 kB 1.3 MB/s eta 0:00:01
     ------------- ----------------------- 92.2/246.4 kB 871.5 kB/s eta 0:00:01
     ---------------- ------------------- 112.6/246.4 kB 726.2 kB/s eta 0:00:01
     ------------------------------- ------ 204.8/246.4 kB 1.0 MB/s eta 0:00:01
     -------------------------------------  245.8/246.4 kB 1.1 MB/s eta 0:00:01
     -------------------------------------  245.8/246.4 kB 1.1 MB/s eta 0:00:01
     ------------------------------------ 246.4/246.4 kB 795.0 kB/s eta 0:00:00
     ---------------------------------------- 0.0/244.9 kB ? eta -:--:--
     ------------------------------ ------- 194.6/244.9 kB 5.9 MB/s eta 0:00:01
     ------------------------------ ------- 194.6/244.9 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: C:\Program Files\Python310\python.exe -m pip install --upgrade pip


In [ ]:
#  Install Brightness Control  
!pip install screen-brightness-control

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/9.6 MB 2.0 MB/s eta 0:00:05
     ---------------------------------------- 0.1/9.6 MB 825.8 kB/s eta 0:00:12
      --------------------------------------- 0.2/9.6 MB 1.4 MB/s eta 0:00:07
      --------------------------------------- 0.2/9.6 MB 1.2 MB/s eta 0:00:08
     - -------------------------------------- 0.3/9.6 MB 1.4 MB/s eta 0:00:07
     - -------------------------------------- 0.5/9.6 MB 2.0 MB/s eta 0:00:05
     -- ------------------------------------- 0.6/9.6 MB 2.0 MB/s eta 0:00:05
     --- ------------------------------------ 0.8/9.6 MB 2.7 MB/s eta 0:00:04
     ---- ----------------------------------- 1.1/9.6 MB 3.0 MB/s eta 0:00:03
     ----- ---------------------------------- 1.3/9.6 MB 3.2 MB/s eta 0:00:03
     ----- ---------------------------------- 1.3/9.6 MB 3.2 MB/s eta

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: C:\Program Files\Python310\python.exe -m pip install --upgrade pip


In [ ]:
# IMPORTS
import cv2
import time
import mediapipe as mp
import math
import numpy as np
import platform
import subprocess
import sys

"""
ULTIMATE HAND GESTURE CONTROLLER — UPDATED
Controls:
- ✋ Thumb Only UP/DOWN → Brightness Up/Down (uses orientation of the thumb)
- ☝️ Index Only (swipe vertically) → Volume Up/Down
- ✌️ Two Fingers (index+middle) → Next Track
- 🤘 Rock On (thumb+index+pinky) → Mute Toggle
- 💞 Pinch (thumb–index):
    • Right hand → Volume (0–100%)
    • Left hand  → Play/Pause

Platforms: Windows, macOS, Linux

Notes:
- macOS brightness requires: brew install brightness
- Linux media controls require: playerctl (MPRIS)
- Windows media keys: optional 'keyboard' module; otherwise ctypes fallback is used
"""

# ===== CONFIGURATION CONSTANTS =====
FRAME_WIDTH, FRAME_HEIGHT = 640, 480
ACTION_COOLDOWN = 0.5            # Minimum seconds between triggering actions
SWIPE_THRESHOLD = 20             # Vertical pixels to count as a swipe step

# Brightness range for safety (avoid totally dark screens)
MIN_BRIGHTNESS = 20
MAX_BRIGHTNESS = 100

# ===== PLATFORM DETECTION =====
CURRENT_OS = platform.system().lower()
print(f"Detected OS: {CURRENT_OS.capitalize()}")

# ===== GESTURE DETECTION UTILITIES =====
def get_finger_status(hand_landmarks, handed_label: str):
    """
    Return [thumb, index, middle, ring, pinky] with 1=extended, 0=folded.
    Uses handedness ('Left'/'Right') to interpret thumb correctly.
    """
    lm = hand_landmarks.landmark
    fingers = []

    # Thumb: compare tip vs IP along x; flip for left/right hand.
    # Indices: 4 (tip), 3 (IP), 2 (MCP)
    if handed_label == 'Right':
        thumb_open = lm[4].x < lm[3].x
    else:  # Left
        thumb_open = lm[4].x > lm[3].x
    fingers.append(1 if thumb_open else 0)

    # Other fingers: tip above PIP => extended (screen coords: up = smaller y)
    for tip, pip in [(8, 6), (12, 10), (16, 14), (20, 18)]:
        fingers.append(1 if lm[tip].y < lm[pip].y else 0)

    return fingers  # [thumb, index, middle, ring, pinky]

def is_two_fingers_up(f):
    # index + middle only
    return f[1] == 1 and f[2] == 1 and f[3] == 0 and f[4] == 0

def is_rock_on(f):
    # thumb + index + pinky up; middle & ring down
    return f[0] == 1 and f[1] == 1 and f[2] == 0 and f[3] == 0 and f[4] == 1

def is_index_only(f):
    # only index up
    return f == [0, 1, 0, 0, 0]

def is_thumb_only(f):
    # only thumb up
    return f[0] == 1 and sum(f[1:]) == 0

def thumb_direction(hand_landmarks) -> str:
    """Return 'up' if thumb tip is above MCP (screen coords), else 'down'."""
    tip = hand_landmarks.landmark[4]
    mcp = hand_landmarks.landmark[2]
    return 'up' if tip.y < mcp.y else 'down'

def get_index_tip_position(hand_landmarks, w, h):
    """Index fingertip in pixel coordinates (x, y)."""
    x = hand_landmarks.landmark[8].x * w
    y = hand_landmarks.landmark[8].y * h
    return x, y

# ===== AUDIO, BRIGHTNESS & MEDIA CONTROLLERS =====
class SystemController:
    """
    Wraps per-OS system controls for volume, mute, brightness, and media keys.
    Some features require extra utilities (documented above).
    """
    def __init__(self):
        self.os = CURRENT_OS
        self.audio_available = False
        self.brightness_control = None
        self.brightness_cli = False
        self.brightness = 50  # local tracker so step-ups work even if we can't read real brightness
        self.setup_controllers()

    # ---------- Setup per OS ----------
    def setup_controllers(self):
        print("Setting up system controls...")
        if self.os == "windows":
            # Audio via pycaw
            try:
                from ctypes import cast, POINTER
                from comtypes import CLSCTX_ALL
                from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
                devices = AudioUtilities.GetSpeakers()
                interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
                self.audio = cast(interface, POINTER(IAudioEndpointVolume))
                self.audio_available = True
                print("Windows audio control: OK (pycaw)")
            except Exception as e:
                print(f"Windows audio not available: {e} | Install: pip install comtypes pycaw")
                self.audio_available = False

            # Brightness via screen_brightness_control
            try:
                import screen_brightness_control as sbc
                self.brightness_control = sbc
                # Try to read current brightness to sync local tracker
                try:
                    current = sbc.get_brightness()
                    if isinstance(current, list) and current:
                        self.brightness = int(current[0])
                    elif isinstance(current, int):
                        self.brightness = current
                except Exception:
                    pass
                print("Brightness control: OK (screen_brightness_control)")
            except Exception as e:
                print(f"Brightness control not available: {e} | Install: pip install screen-brightness-control")
                self.brightness_control = None

            # Windows media keys (optional 'keyboard' module)
            try:
                import keyboard  # optional
                self._keyboard = keyboard
                print("Windows media keys: OK (keyboard module)")
            except Exception:
                self._keyboard = None
                print("Windows media keys: using ctypes fallback")

        elif self.os == "darwin":  # macOS
            # Audio via osascript
            try:
                subprocess.run(['osascript', '-e', 'output volume of (get volume settings)'],
                               capture_output=True, text=True)
                self.audio_available = True
                print("macOS audio control: OK (osascript)")
            except Exception as e:
                self.audio_available = False
                print(f"macOS audio control not available: {e}")

            # brightness CLI
            try:
                subprocess.run(['brightness', '-h'], capture_output=True)
                self.brightness_cli = True
                print("macOS brightness: OK (brightness tool)")
            except Exception:
                self.brightness_cli = False
                print("Install macOS brightness tool: brew install brightness")

        elif self.os == "linux":
            # Audio via amixer
            self.audio_available = subprocess.run(['which', 'amixer'], capture_output=True).returncode == 0
            print("Linux audio: OK (amixer)" if self.audio_available else "Linux audio: install amixer")

            # Brightness via brightnessctl
            self.brightness_available = subprocess.run(['which', 'brightnessctl'], capture_output=True).returncode == 0
            print("Linux brightness: OK (brightnessctl)" if self.brightness_available else "Install brightnessctl")

            # Media via playerctl
            self.playerctl_available = subprocess.run(['which', 'playerctl'], capture_output=True).returncode == 0
            print("Linux media: OK (playerctl)" if self.playerctl_available else "Install playerctl for media keys")

    # ---------- Volume ----------
    def get_volume(self) -> int:
        """Return system volume as percentage [0..100]."""
        try:
            if self.os == "windows" and self.audio_available:
                vol = self.audio.GetMasterVolumeLevelScalar()
                return int(round(vol * 100))
            elif self.os == "darwin":
                result = subprocess.run(['osascript', '-e', 'output volume of (get volume settings)'],
                                        capture_output=True, text=True)
                return int(result.stdout.strip())
            elif self.os == "linux" and self.audio_available:
                result = subprocess.run(['amixer', 'get', 'Master'], capture_output=True, text=True)
                import re
                match = re.search(r'(\d+)%', result.stdout)
                return int(match.group(1)) if match else 50
        except Exception:
            pass
        return 50  # Reasonable default if reading fails

    def set_volume(self, vol: int):
        """Set system volume to percentage [0..100]."""
        vol = max(0, min(100, int(vol)))
        try:
            if self.os == "windows" and self.audio_available:
                self.audio.SetMasterVolumeLevelScalar(vol / 100.0, None)
            elif self.os == "darwin":
                subprocess.run(['osascript', '-e', f'set volume output volume {vol}'])
            elif self.os == "linux" and self.audio_available:
                subprocess.run(['amixer', '-q', 'set', 'Master', f'{vol}%'])
        except Exception:
            pass

    def increase_volume(self, step=5):
        """Step volume up by 'step' percent."""
        self.set_volume(self.get_volume() + step)

    def decrease_volume(self, step=5):
        """Step volume down by 'step' percent."""
        self.set_volume(self.get_volume() - step)

    def toggle_mute(self):
        """Toggle system mute."""
        try:
            if self.os == "windows" and self.audio_available:
                muted = self.audio.GetMute()
                self.audio.SetMute(0 if muted else 1, None)
            elif self.os == "darwin":
                get_muted = subprocess.run(
                    ['osascript', '-e', 'output muted of (get volume settings)'],
                    capture_output=True, text=True
                ).stdout.strip().lower()
                new_state = 'false' if get_muted == 'true' else 'true'
                subprocess.run(['osascript', '-e', f'set volume output muted {new_state}'])
            elif self.os == "linux":
                subprocess.run(['amixer', '-q', 'set', 'Master', 'toggle'])
        except Exception:
            pass

    # ---------- Brightness ----------
    def set_brightness(self, level: int):
        """Set brightness to [MIN_BRIGHTNESS..MAX_BRIGHTNESS]."""
        level = max(MIN_BRIGHTNESS, min(MAX_BRIGHTNESS, int(level)))
        self.brightness = level  # Keep local state in sync even if back-end fails
        try:
            if self.os == "windows" and self.brightness_control:
                self.brightness_control.set_brightness(level)
            elif self.os == "darwin" and self.brightness_cli:
                subprocess.run(['brightness', str(level / 100.0)])
            elif self.os == "linux" and getattr(self, 'brightness_available', False):
                subprocess.run(['brightnessctl', 'set', f'{level}%'])
        except Exception:
            pass

    def increase_brightness(self, step=10):
        """Step brightness up by 'step' percent."""
        self.set_brightness(self.brightness + step)

    def decrease_brightness(self, step=10):
        """Step brightness down by 'step' percent."""
        self.set_brightness(self.brightness - step)

    # ---------- Media Controls ----------
    def play_pause(self):
        """Send play/pause to the default media player."""
        try:
            if self.os == "windows":
                if getattr(self, '_keyboard', None):
                    self._keyboard.send('play/pause media')
                else:
                    # ctypes fallback: keybd_event with VK_MEDIA_PLAY_PAUSE (0xB3)
                    import ctypes
                    VK_MEDIA_PLAY_PAUSE = 0xB3
                    KEYEVENTF_KEYUP = 0x0002
                    ctypes.windll.user32.keybd_event(VK_MEDIA_PLAY_PAUSE, 0, 0, 0)
                    ctypes.windll.user32.keybd_event(VK_MEDIA_PLAY_PAUSE, 0, KEYEVENTF_KEYUP, 0)
            elif self.os == "darwin":
                # Prefer controlling apps directly
                for app, cmd in [("Music", "playpause"), ("Spotify", "playpause")]:
                    try:
                        subprocess.run(['osascript', '-e', f'tell application "{app}" to {cmd}'],
                                       capture_output=True, text=True)
                        return
                    except Exception:
                        continue
            elif self.os == "linux":
                if getattr(self, 'playerctl_available', False):
                    subprocess.run(['playerctl', 'play-pause'])
        except Exception:
            pass

    def next_track(self):
        """Go to next track in the default media player."""
        try:
            if self.os == "windows":
                if getattr(self, '_keyboard', None):
                    self._keyboard.send('next track')
                else:
                    import ctypes
                    VK_MEDIA_NEXT_TRACK = 0xB0
                    KEYEVENTF_KEYUP = 0x0002
                    ctypes.windll.user32.keybd_event(VK_MEDIA_NEXT_TRACK, 0, 0, 0)
                    ctypes.windll.user32.keybd_event(VK_MEDIA_NEXT_TRACK, 0, KEYEVENTF_KEYUP, 0)
            elif self.os == "darwin":
                for app, cmd in [("Music", "next track"), ("Spotify", "next track")]:
                    try:
                        subprocess.run(['osascript', '-e', f'tell application "{app}" to {cmd}'],
                                       capture_output=True, text=True)
                        return
                    except Exception:
                        continue
            elif self.os == "linux":
                if getattr(self, 'playerctl_available', False):
                    subprocess.run(['playerctl', 'next'])
        except Exception:
            pass

# ===== INITIALIZATION =====
print("\n" + "="*50)
print("   ULTIMATE HAND GESTURE CONTROLLER")
print("="*50)

controller = SystemController()

# Runtime state
last_action_time = 0.0   # last time an action was fired (cooldown gate)
last_swipe_y = None      # last Y position of index finger for swipe stepping
gesture_feedback = ""    # short on-screen status text
feedback_timer = 0.0     # when the feedback was last updated

# ===== CAMERA & MEDIAPIPE SETUP =====
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open camera")
    sys.exit(1)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

# MediaPipe Hands pipeline
with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
) as hands:

    while True:
        success, img = cap.read()
        if not success:
            continue

        # Flip for mirror view; convert to RGB for MediaPipe
        img = cv2.flip(img, 1)
        h, w = img.shape[:2]
        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        rgb.flags.writeable = False
        results = hands.process(rgb)
        rgb.flags.writeable = True

        current_time = time.time()
        vol = controller.get_volume()
        # Map volume [0..100] to a vertical bar between 20% and 85% of frame height
        volbar = int(np.interp(vol, [0, 100], [int(0.85*h), int(0.2*h)]))

        # ===== DRAW SIMPLE UI =====
        cv2.rectangle(img, (50, int(0.2*h)), (85, int(0.85*h)), (153, 204, 255), 3)
        cv2.rectangle(img, (50, volbar), (85, int(0.85*h)), (102, 255, 255), cv2.FILLED)
        cv2.putText(img, f'{vol}%', (40, 40), cv2.FONT_HERSHEY_COMPLEX, 0.7, (26, 26, 26), 2)
        cv2.putText(img, f'OS: {CURRENT_OS.capitalize()}', (10, 25),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 150, 255), 2)

        if gesture_feedback and current_time < feedback_timer + 1.8:
            cv2.putText(img, gesture_feedback, (200, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)

        # ===== PROCESS HANDS =====
        if results.multi_hand_landmarks:
            # Pair landmarks with handedness labels (fallback to 'Right' if missing)
            handed_list = results.multi_handedness or []
            pairs = list(zip(results.multi_hand_landmarks,
                             handed_list + [None] * (len(results.multi_hand_landmarks) - len(handed_list))))

            for hand_landmarks, handedness in pairs:
                # Draw landmarks for visual feedback
                mp_drawing.draw_landmarks(
                    img, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=3),
                    mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=2, circle_radius=2)
                )

                label = 'Right'
                if handedness is not None and handedness.classification:
                    label = handedness.classification[0].label  # 'Left' or 'Right'

                fingers = get_finger_status(hand_landmarks, label)
                ix, iy = get_index_tip_position(hand_landmarks, w, h)

                # Cooldown gate
                can_act = (current_time - last_action_time) > ACTION_COOLDOWN

                # ---- THUMB ONLY: Brightness Up/Down (by orientation) ----
                if can_act and is_thumb_only(fingers):
                    direction = thumb_direction(hand_landmarks)
                    if direction == 'up':
                        controller.increase_brightness()
                        gesture_feedback = "Brightness Up"
                    else:
                        controller.decrease_brightness()
                        gesture_feedback = "Brightness Down"
                    feedback_timer = current_time
                    last_action_time = current_time

                # ---- TWO FINGERS: Next Track ----
                elif can_act and is_two_fingers_up(fingers):
                    controller.next_track()
                    gesture_feedback = "Next Track"
                    feedback_timer = current_time
                    last_action_time = current_time

                # ---- ROCK ON: Mute Toggle ----
                elif can_act and is_rock_on(fingers):
                    controller.toggle_mute()
                    gesture_feedback = "Mute Toggled"
                    feedback_timer = current_time
                    last_action_time = current_time

                # ---- INDEX-ONLY SWIPE: Volume Up/Down ----
                if is_index_only(fingers):
                    if last_swipe_y is not None:
                        dy = last_swipe_y - iy
                        if abs(dy) > SWIPE_THRESHOLD and (current_time - last_action_time) > 0.1:
                            if dy > 0:
                                controller.increase_volume()
                                gesture_feedback = "Volume Up"
                            else:
                                controller.decrease_volume()
                                gesture_feedback = "Volume Down"
                            feedback_timer = current_time
                            last_swipe_y = iy
                            last_action_time = current_time
                    else:
                        last_swipe_y = iy
                else:
                    last_swipe_y = None

                # ---- PINCH (thumb–index) ----
                x1 = int(hand_landmarks.landmark[4].x * w)
                y1 = int(hand_landmarks.landmark[4].y * h)
                x2 = int(hand_landmarks.landmark[8].x * w)
                y2 = int(hand_landmarks.landmark[8].y * h)
                length = math.hypot(x2 - x1, y2 - y1)

                # Practical range so you actually get 0..100%
                if 20 <= length <= 200:
                    # Visualize pinch
                    cv2.circle(img, ((x1 + x2) // 2, (y1 + y2) // 2), 10, (0, 255, 0), cv2.FILLED)
                    cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 3)

                    if (current_time - last_action_time) > 0.05:
                        if label == 'Right':
                            vol_val = np.interp(length, [20, 200], [0, 100])
                            controller.set_volume(int(vol_val))
                            gesture_feedback = f"Vol: {int(vol_val)}%"
                        else:  # Left hand -> Play/Pause
                            controller.play_pause()
                            gesture_feedback = "Play/Pause"
                        feedback_timer = current_time
                        last_action_time = current_time

        cv2.imshow('ULTIMATE GESTURE CONTROL', img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# ===== CLEANUP =====
cap.release()
cv2.destroyAllWindows()
print("Gesture controller stopped.")

Detected OS: Windows

   ULTIMATE HAND GESTURE CONTROLLER
Setting up system controls...
Windows audio control: OK (pycaw)
Brightness control not available: No module named 'screen_brightness_control' | Install: pip install screen-brightness-control
Windows media keys: using ctypes fallback
Gesture controller stopped.
